In [1]:
import os
import time
import smtplib
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

# Email configuration settings
SMTP_SERVER = 'smtp.gmail.com'  # The SMTP server for sending emails (Gmail in this case)
SMTP_PORT = 587  # The port used for TLS
SENDER_EMAIL = 'attaluri.koushik7@gmail.com'  # Your Gmail address (sender)
SENDER_PASSWORD = 'kvoi hpds dvpz mfem'  # Your App Password generated from Gmail
RECIPIENT_EMAIL = 'blamedsheep74@gmail.com'  # Recipient's email address

# Directory to monitor for new files
DIRECTORY_TO_WATCH = 'C:\\Test'  # Replace this with the path to the directory you want to monitor

class Watcher(FileSystemEventHandler):
    """
    A class that handles file system events.
    In this case, it will send an email when a new file is created.
    """

    def on_created(self, event):
        """
        Method called when a file or directory is created.            
        It sends an email if a new file is created.
        
        :param event: The file system event.
        """
        # Check if the event is not a directory and the file does not start with '~'
        if not event.is_directory and not os.path.basename(event.src_path).startswith('~'):
            self.send_email(event.src_path)

    def send_email(self, file_path):
                                                                        
        try:
            # Set up the MIME structure of the email
            msg = MIMEMultipart()
            msg['From'] = SENDER_EMAIL
            msg['To'] = RECIPIENT_EMAIL
            msg['Subject'] = 'New File Detected'

            # Email body text
            body = 'A new file has been detected. Please find the attached file.'
            msg.attach(MIMEText(body, 'plain'))

            # Attach the new file to the email
            with open(file_path, 'rb') as attachment:
                part = MIMEBase('application', 'octet-stream')
                part.set_payload(attachment.read())
                encoders.encode_base64(part)  # Encode the file in base64
                part.add_header('Content-Disposition', f'attachment; filename={os.path.basename(file_path)}')
                msg.attach(part)

            # Establish a secure SMTP connection
            server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
            server.starttls()  # Upgrade the connection to a secure encrypted SSL/TLS connection
            server.login(SENDER_EMAIL, SENDER_PASSWORD)  # Log in to the email server
            text = msg.as_string()  # Convert the message to a string format
            server.sendmail(SENDER_EMAIL, RECIPIENT_EMAIL, text)  # Send the email
            server.quit()  # Terminate the SMTP session
            print(f"Email sent successfully with file: {file_path}")

        except PermissionError as e:
            print(f"Permission denied for file: {file_path}. Error: {e}")
        except Exception as e:
            print(f"Failed to send email. Error: {e}")

def start_monitoring():
    """
    Sets up the directory monitoring and starts the event loop.
    """
    event_handler = Watcher()  # Create an instance of the Watcher class
    observer = Observer()  # Create an observer that will watch the directory
    observer.schedule(event_handler, DIRECTORY_TO_WATCH, recursive=False)  # Monitor the specified directory
    observer.start()  # Start the observer
    print(f"Monitoring directory: {DIRECTORY_TO_WATCH}")

    try:
        while True:
            time.sleep(1)  # Keep the observer running
    except KeyboardInterrupt:
        observer.stop()  # Stop the observer if interrupted
    observer.join()  # Wait for the observer thread to finish

if __name__ == "__main__":
    start_monitoring()  # Start monitoring when the script is run

